## 1. PP-HumanSegV2模型简介

将人物和背景在像素级别进行区分，是一个图像分割的经典任务，具有广泛的应用。 一般而言，该任务可以分为两类：针对半身人像的分割，简称肖像分割；针对全身和半身人像的分割，简称通用人像分割。

对于肖像分割和通用人像分割，PaddleSeg发布了PP-HumanSeg系列模型，具有分割精度高、推理速度快、通用型强的优点。而且PP-HumanSeg系列模型可以开箱即用，零成本部署到产品中，也支持针对特定场景数据进行微调，实现更佳分割效果。

2022年7月，PaddleSeg重磅升级的PP-HumanSegV2人像分割方案，以96.63%的mIoU精度， 63FPS的手机端推理速度，再次刷新开源人像分割算法SOTA指标。相比PP-HumanSegV1方案，推理速度提升87.15%，分割精度提升3.03%，可视化效果更佳。V2方案可与商业收费方案媲美，而且支持零成本、开箱即用！

PP-HumanSeg由飞桨官方出品，是PaddleSeg团队推出的模型和方案。 更多关于PaddleSeg可以点击 https://github.com/PaddlePaddle/PaddleSeg 进行了解。

## 2. 模型效果及应用场景
### 2.1 肖像分割和通用人像分割任务

#### 2.1.1 数据集

数据集以PP-HumanSeg14k为主，分为训练集和测试集。

#### 2.1.2 模型效果速览

PP-HumanSegV2在图片上的分割效果如下。

原图：
<div align="center">
<img src="https://user-images.githubusercontent.com/48357642/200734740-72e98c73-5b41-47c4-b208-7fd9c10d8b8a.jpeg"  width = "60%"  />
</div>


分割后的图：
<div align="center">
<img src="https://user-images.githubusercontent.com/48357642/200735017-eb8a2b22-7ef9-4e4f-acc2-1ea538672f75.jpeg"  width = "60%"  />
</div>


## 3. 模型如何使用

### 3.1 模型推理



* 安装PaddlePaddle

安装PaddlePaddle，要求PaddlePaddle >= 2.2.0。由于图像分割模型计算开销大，推荐在GPU版本的PaddlePaddle下使用。

在AIStudio中，大家选择可以直接选择安装好PaddlePaddle的环境。
如果需要执行安装PaddlePaddle，请参考[PaddlePaddle官网](https://www.paddlepaddle.org.cn/install/quick?docurl=/documentation/docs/zh/install/pip/linux-pip.html)。

本教程在PaddlePaddle 2.3.2版本下进行了验证。


* 下载PaddleSeg 

（不在Jupyter Notebook上运行时需要将"！"或者"%"去掉。）

In [3]:
%cd ~

!git clone https://gitee.com/PaddlePaddle/PaddleSeg.git

/home/aistudio
正克隆到 'PaddleSeg'...
remote: Enumerating objects: 20171, done.
remote: Counting objects: 100% (5140/5140), done.
remote: Compressing objects: 100% (2425/2425), done.
remote: Total 20171 (delta 3243), reused 4335 (delta 2647), pack-reused 15031
接收对象中: 100% (20171/20171), 345.54 MiB | 1.46 MiB/s, 完成.
处理 delta 中: 100% (13103/13103), 完成.
检查连接... 完成。


* 安装PaddleSeg

In [4]:
# 安装PaddleSeg
%cd ~/PaddleSeg
!pip install -v -e .

/home/aistudio/PaddleSeg
Using pip 22.1.2 from /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pip (python 3.7)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///home/aistudio/PaddleSeg
  Preparing metadata (setup.py) ...   Running command python setup.py egg_info
  /home/aistudio/PaddleSeg/paddleseg/models/losses/rmi_loss.py:78: DeprecationWarning: invalid escape sequence \i
    """
  running egg_info
  creating /tmp/pip-pip-egg-info-gc0tcpql/paddleseg.egg-info
  writing /tmp/pip-pip-egg-info-gc0tcpql/paddleseg.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-gc0tcpql/paddleseg.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-gc0tcpql/paddleseg.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-gc0tcpql/paddleseg.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-gc0tcpql/paddleseg.egg-info/SOURCES.txt'
  adding license file 'LICENSE' (matched patt

* 下载数据和模型

In [5]:
%cd ~/PaddleSeg/contrib/PP-HumanSeg
!python src/download_inference_models.py
!python src/download_data.py

/home/aistudio/PaddleSeg/contrib/PP-HumanSeg
Connecting to https://paddleseg.bj.bcebos.com/dygraph/pp_humanseg_v2/portrait_pp_humansegv1_lite_398x224_inference_model_with_softmax.zip
[==================================================] 100.00%
Uncompress portrait_pp_humansegv1_lite_398x224_inference_model_with_softmax.zip
[==================================================] 100.00%
Connecting to https://paddleseg.bj.bcebos.com/dygraph/pp_humanseg_v2/portrait_pp_humansegv2_lite_256x144_smaller/portrait_pp_humansegv2_lite_256x144_inference_model_with_softmax.zip
[==================================================] 100.00%
Uncompress portrait_pp_humansegv2_lite_256x144_inference_model_with_softmax.zip
[==================================================] 100.00%
Connecting to https://paddleseg.bj.bcebos.com/dygraph/pp_humanseg_v2/human_pp_humansegv1_lite_192x192_inference_model_with_softmax.zip
[==================================================] 100.00%
Uncompress human_pp_humansegv1_lite

* 快速体验


In [6]:
!python src/seg_demo.py \
  --config inference_models/portrait_pp_humansegv2_lite_256x144_inference_model_with_softmax/deploy.yaml \
  --img_path data/images/portrait_heng.jpg \
  --bg_img_path data/images/bg_2.jpg \
  --save_dir data/images_result/portrait_heng_v2_withbg.jpg

!python src/seg_demo.py \
  --config inference_models/portrait_pp_humansegv2_lite_256x144_inference_model_with_softmax/deploy.yaml \
  --img_path data/images/portrait_shu.jpg \
  --bg_img_path data/images/bg_1.jpg \
  --save_dir data/images_result/portrait_shu_v2_withbg.jpg \
  --vertical_screen

2022-11-09 17:38:28 [INFO]	Input: image
2022-11-09 17:38:28 [INFO]	Create predictor...
2022-11-09 17:38:29 [INFO]	Start predicting...
2022-11-09 17:38:33 [INFO]	Input: image
2022-11-09 17:38:33 [INFO]	Create predictor...
2022-11-09 17:38:35 [INFO]	Start predicting...


结果保存在`data/images_result/portrait_heng_v2.jpg`（如下图）。

<img src="https://user-images.githubusercontent.com/52520497/188776878-130f4f6a-6379-4fb0-87e4-9a7ee4707c1d.jpg" width="200"> 


### 3.2 模型训练


* 准备

参考前文，安装PaddleSeg、下载数据集，然后下载预训练权重。

In [7]:
!python src/download_pretrained_models.py

Data download finished!
Connecting to https://paddleseg.bj.bcebos.com/dygraph/pp_humanseg_v2/human_pp_humansegv1_lite_192x192_pretrained.zip
[==================================================] 100.00%
Uncompress human_pp_humansegv1_lite_192x192_pretrained.zip
[==================================================] 100.00%
Connecting to https://paddleseg.bj.bcebos.com/dygraph/pp_humanseg_v2/human_pp_humansegv2_lite_192x192_pretrained.zip
[==================================================] 100.00%
Uncompress human_pp_humansegv2_lite_192x192_pretrained.zip
[==================================================] 100.00%
Connecting to https://paddleseg.bj.bcebos.com/dygraph/pp_humanseg_v2/human_pp_humansegv1_mobile_192x192_pretrained.zip
[==================================================] 100.00%
Uncompress human_pp_humansegv1_mobile_192x192_pretrained.zip
[==================================================] 100.00%
Connecting to https://paddleseg.bj.bcebos.com/dygraph/pp_humanseg_v2/human_pp_

* 训练

配置文件保存在`./configs`目录下，如下。配置文件中，已经通过`pretrained`设置好预训练权重的路径。
```
configs
├── human_pp_humansegv1_lite.yml
├── human_pp_humansegv2_lite.yml
├── human_pp_humansegv1_mobile.yml
├── human_pp_humansegv2_mobile.yml
├── human_pp_humansegv1_server.yml
├── portrait_pp_humansegv1_lite.yml
├── portrait_pp_humansegv2_lite.yml
```

执行如下命令，进行模型微调（大家需要根据实际情况修改配置文件中的超参）。模型训练的详细文档，请参考[链接](https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.6/docs/train/train_cn.md)。

In [8]:
!export CUDA_VISIBLE_DEVICES=0 # Linux下设置1张可用的卡
# set CUDA_VISIBLE_DEVICES=0  # Windows下设置1张可用的卡
!python ../../train.py \
  --config configs/human_pp_humansegv2_lite.yml \
  --save_dir output/human_pp_humansegv2_lite \
  --save_interval 100 --do_eval --use_vdl

2022-11-09 17:42:54 [INFO]	
------------Environment Information-------------
platform: Linux-4.15.0-140-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Build cuda_11.2.r11.2/compiler.29618528_0
cudnn: 8.2
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-16GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddleSeg: 2.6.0
PaddlePaddle: 2.3.2
OpenCV: 4.1.1
------------------------------------------------
2022-11-09 17:42:54 [INFO]	
---------------Config Information---------------
batch_size: 8
export:
  transforms:
  - target_size:
    - 192
    - 192
    type: Resize
  - type: Normalize
iters: 1000
loss:
  coef:
  - 1
  - 1
  - 1
  - 1
  types:
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.0001
  power: 0.9
  type: PolynomialDecay
model:
  arm_out_chs:
  - 32
  

* 评估

执行如下命令，加载模型和训练好的权重，进行模型评估，输出验证集上的评估精度。模型评估的详细文档，请参考[链接](https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.6/docs/evaluation/evaluate/evaluate_cn.md)。

In [9]:
!python ../../val.py \
  --config configs/human_pp_humansegv2_lite.yml \
  --model_path pretrained_models/human_pp_humansegv2_lite_192x192_pretrained/model.pdparams

2022-11-09 17:43:26 [INFO]	
---------------Config Information---------------
batch_size: 8
export:
  transforms:
  - target_size:
    - 192
    - 192
    type: Resize
  - type: Normalize
iters: 1000
loss:
  coef:
  - 1
  - 1
  - 1
  - 1
  types:
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.0001
  power: 0.9
  type: PolynomialDecay
model:
  arm_out_chs:
  - 32
  - 64
  - 96
  - 128
  backbone:
    pretrained: https://paddleseg.bj.bcebos.com/dygraph/backbone/mobilenetv3_large_x1_0_ssld.tar.gz
    type: MobileNetV3_large_x1_0
  backbone_indices:
  - 0
  - 1
  - 2
  - 3
  cm_bin_sizes:
  - 1
  - 2
  - 4
  cm_out_ch: 128
  num_classes: 2
  pretrained: pretrained_models/human_pp_humansegv2_lite_192x192_pretrained/model.pdparams
  seg_head_inter_chs:
  - 16
  - 32
  - 32
  - 32
  type: MobileSeg
  use_last_fuse: true
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 0.

* 预测

执行如下命令，加载模型和训练好的权重，对单张图像进行预测，预测结果保存在`./data/images_result`目录下的`added_prediction`和`pseudo_color_prediction`文件夹中。

In [10]:
!python ../../predict.py \
  --config configs/human_pp_humansegv2_lite.yml \
  --model_path pretrained_models/human_pp_humansegv2_lite_192x192_pretrained/model.pdparams \
  --image_path data/images/human.jpg \
  --save_dir ./data/images_result

2022-11-09 17:43:43 [INFO]	
---------------Config Information---------------
batch_size: 8
export:
  transforms:
  - target_size:
    - 192
    - 192
    type: Resize
  - type: Normalize
iters: 1000
loss:
  coef:
  - 1
  - 1
  - 1
  - 1
  types:
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.0001
  power: 0.9
  type: PolynomialDecay
model:
  arm_out_chs:
  - 32
  - 64
  - 96
  - 128
  backbone:
    pretrained: https://paddleseg.bj.bcebos.com/dygraph/backbone/mobilenetv3_large_x1_0_ssld.tar.gz
    type: MobileNetV3_large_x1_0
  backbone_indices:
  - 0
  - 1
  - 2
  - 3
  cm_bin_sizes:
  - 1
  - 2
  - 4
  cm_out_ch: 128
  num_classes: 2
  pretrained: pretrained_models/human_pp_humansegv2_lite_192x192_pretrained/model.pdparams
  seg_head_inter_chs:
  - 16
  - 32
  - 32
  - 32
  type: MobileSeg
  use_last_fuse: true
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 0.

* 导出

执行如下命令，加载模型和训练好的权重，导出预测模型。模型导出的详细文档，请参考[链接](https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.6/docs/model_export_cn.md)。

In [11]:
!python ../../export.py \
  --config configs/human_pp_humansegv2_lite.yml \
  --model_path pretrained_models/human_pp_humansegv2_lite_192x192_pretrained/model.pdparams \
  --save_dir output/human_pp_humansegv2_lite \
  --without_argmax \
  --with_softmax

W1109 17:44:01.011113  8827 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W1109 17:44:01.015580  8827 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.
2022-11-09 17:44:02 [INFO]	Loading pretrained model from https://paddleseg.bj.bcebos.com/dygraph/backbone/mobilenetv3_large_x1_0_ssld.tar.gz
2022-11-09 17:44:02 [INFO]	There are 262/262 variables loaded into MobileNetV3.
2022-11-09 17:44:02 [INFO]	Loading pretrained model from pretrained_models/human_pp_humansegv2_lite_192x192_pretrained/model.pdparams
2022-11-09 17:44:02 [INFO]	There are 471/471 variables loaded into MobileSeg.
2022-11-09 17:44:02 [INFO]	Loaded trained params of model successfully.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:341: UserWarning: /tmp/tmp2axz6dfp.py:26
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code 

注意，使用--without_argmax --with_softmax参数，则模型导出的时候，模型最后面不会添加Argmax算子，而是添加Softmax算子。 所以，输出是浮点数类型，表示前景的概率，使得图像融合的边缘更为平滑。

## 4. 模型原理

模型结构如下图。
<div align="center">
<img src="https://user-images.githubusercontent.com/48357642/200757494-1e63215e-4cd1-4c39-8dd9-a0e37f8719f2.png"  width = "60%"  />
</div>

* 模型算量大幅减小

对于模型Encoder部分，我们选用MobileNetV3作为骨干网络提取多层特征，分析发现MobileNetV3的参数主要集中在最后一个Stage，在不影响分割精度的前提下，我们只保留MobileNetV3的前四个Stage，成功减少了68.6%的参数量。对于上下文部分，我们使用PP-LiteSeg模型中提出的轻量级SPPM模块，而且其中的普通卷积都替换为可分离卷积，进一步减小计算量。对于Decoder部分，我们设计三个Fusion融合模块，多次融合深层语义特征和浅层细节特征，最后一个Fusion融合模块再次汇集不同层次的特征图，输出分割结果。

多层次特征融合模块图：
<div align="center">
<img src="https://user-images.githubusercontent.com/48357642/200758284-a8d5e6f9-1a66-414b-804c-57c6b6fcd698.png"  width = "30%"  />
</div>

* 使用两阶段训练方式，提升分割精度

两阶段训练是基于迁移学习的思想，首先在大规模混合人像数据集（数据量100k+）上训练，然后使用该预训练权重，在PP-HumanSeg14k数据集（数据量14k）上训练，最终得到训练好的模型。使用两阶段训练方式，可以充分利用其他数据集，提高模型的分割精度和泛化能力。

* 调整图像分辨率，提升推理速度

调整图像分辨率也直接影响模型的推理速度，我们使用多种图像分辨率进行训练和测试，在PP-HumanSeg v2方案中选择最佳图像分辨率，进一步提升了模型推理速度。

* 使用形态学后处理，提升可视化效果

首先获取原始预测图像I，然后使用阈值处理、图像腐蚀、图像膨胀等操作得到掩码图像M，最后预测图像I和掩码图像M相乘，输出最终预测图像O。


## 5. 相关论文以及引用信息
如果我们的项目在学术上帮助到你，请考虑以下引用：

```
@InProceedings{Chu_2022_WACV,
    author    = {Chu, Lutao and Liu, Yi and Wu, Zewu and Tang, Shiyu and Chen, Guowei and Hao, Yuying and Peng, Juncai and Yu, Zhiliang and Chen, Zeyu and Lai, Baohua and Xiong, Haoyi},
    title     = {PP-HumanSeg: Connectivity-Aware Portrait Segmentation With a Large-Scale Teleconferencing Video Dataset},
    booktitle = {Proceedings of the IEEE/CVF Winter Conference on Applications of Computer Vision (WACV) Workshops},
    month     = {January},
    year      = {2022},
    pages     = {202-209}
}
```